In [1]:
!pip install pyDOE
!pip install cma

  Created wheel for pyDOE: filename=pyDOE-0.3.8-cp36-none-any.whl size=18178 sha256=22bd684d1eb91b728ffa2cbf5cc776ff0694711dea57c04d74ce66ba275a73d4
  Stored in directory: /root/.cache/pip/wheels/7c/c8/58/a6493bd415e8ba5735082b5e0c096d7c1f2933077a8ce34544
Successfully built pyDOE
     |████████████████████████████████| 245kB 2.8MB/s 


In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
import bbobbenchmarks as bn
import sklearn
from sklearn.preprocessing import PolynomialFeatures

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F3 '''
def F3(X):
    f = bn.F3()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data


''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))


''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 0.9227134488122688 ,random_state=0 , l1_ratio=1.0, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)


''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 37.213462 , C = 1000.000000 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_3_2000Samples.csv"
train = pd.read_csv(path).iloc[:,1:]
test = pd.read_csv('test_3_400Samples.csv').iloc[:,1:]
true = np.array(test['Y'])

## Surrogate Models

In [5]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


## CMA-ES

In [6]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,101):
        Cols.append(Columns[j]+'_X'+str(i))
const = [ [-5] * 100, [5] * 100 ]

opt = cma.CMAOptions()
opt.set("bounds", const)
opt.set ("seed" , 0)
opt.set ("maxfevals" , 100000)

n_obs , dim =  30, 100
G = DOE(n_obs, dim)
G  = [ linearscaletransform(G[idx] , range_out=(-5,5)) for idx in range(dim) ]
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([30,300])
for i in range(G[1].shape[0]):
  print ('Run : '+ str(i))
  min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  X_Values [i,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 0
(8_w,17)-aCMA-ES (mu_w=5.1,w_1=31%) in dimension 100 (seed=956607, Mon Apr 13 21:45:53 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     17 1.044950992114682e+04 1.0e+00 2.40e+00  2e+00  2e+00 0:00.0
    2     34 9.257832929749868e+03 1.0e+00 2.33e+00  2e+00  2e+00 0:00.1
    3     51 8.771180088242370e+03 1.0e+00 2.27e+00  2e+00  2e+00 0:00.1


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [12507.39871038] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [9880.72885921] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


   87   1479 -8.170161232560968e+03 1.3e+00 1.21e+00  1e+00  1e+00 0:03.1
  100   1700 -8.341297895350443e+03 1.3e+00 1.03e+00  1e+00  1e+00 0:03.6
  200   3400 -1.854138639021648e+04 1.4e+00 1.94e-01  2e-01  2e-01 0:07.2
  300   5100 -1.903588681969274e+04 1.4e+00 3.74e-02  3e-02  4e-02 0:10.8
  400   6800 -1.905039344781576e+04 1.4e+00 6.38e-03  6e-03  6e-03 0:14.3
  500   8500 -1.905080217810094e+04 1.5e+00 1.33e-03  1e-03  1e-03 0:17.9
  600  10200 -1.905082812386116e+04 1.6e+00 4.27e-04  3e-04  5e-04 0:21.5
  700  11900 -1.905083067775680e+04 2.0e+00 1.34e-04  1e-04  2e-04 0:25.1
  800  13600 -1.905083088183705e+04 2.3e+00 2.86e-05  2e-05  5e-05 0:28.6
  900  15300 -1.905083089194408e+04 2.4e+00 7.33e-06  5e-06  1e-05 0:32.2
 1000  17000 -1.905083089304344e+04 2.6e+00 3.62e-06  3e-06  6e-06 0:35.8
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  18700 -1.905083089321872e+04 3.1e+00 1.13e-06  8e-07  2e-06 0:39.3
 1200  20400 -1.905083089323253e+04 3.5e

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [12551.07538939] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=12551.075389, sigma=2.40e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=12551.075389, sigma=2.40e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  '

    2     34 8.705074537039272e+03 1.0e+00 2.33e+00  2e+00  2e+00 0:00.1
    3     51 7.756803108967360e+03 1.0e+00 2.28e+00  2e+00  2e+00 0:00.1
   77   1309 -1.460004906777940e+03 1.2e+00 1.41e+00  1e+00  1e+00 0:03.1
  100   1700 -3.696094913175390e+03 1.3e+00 1.12e+00  1e+00  1e+00 0:04.0
  200   3400 -6.885103612696324e+03 1.3e+00 3.62e-01  3e-01  4e-01 0:08.1
  300   5100 -7.753015944932038e+03 1.4e+00 1.25e-01  1e-01  1e-01 0:12.2
  400   6800 -7.892450033892068e+03 1.6e+00 5.74e-02  5e-02  6e-02 0:16.2
  500   8500 -7.952872117806357e+03 2.2e+00 5.64e-02  5e-02  8e-02 0:20.3
  600  10200 -8.047847181567739e+03 3.0e+00 6.47e-02  5e-02  1e-01 0:24.3
  700  11900 -8.323756158956336e+03 3.5e+00 1.22e-01  1e-01  2e-01 0:28.4
  800  13600 -8.599071658836470e+03 4.0e+00 6.03e-02  5e-02  1e-01 0:32.4
  900  15300 -8.633700092465650e+03 3.9e+00 3.18e-02  3e-02  6e-02 0:36.5
 1000  17000 -8.665343995720301e+03 3.9e+00 4.21e-02  3e-02  8e-02 0:40.5
Iterat #Fevals   function value  axis ra

...

   87   1479 -1.010181304559704e+04 1.3e+00 9.41e-01  9e-01  1e+00 0:03.1
  100   1700 -1.287526258909290e+04 1.3e+00 7.42e-01  7e-01  8e-01 0:03.6
  200   3400 -1.883134173717872e+04 1.3e+00 1.42e-01  1e-01  1e-01 0:07.1
  300   5100 -1.904409244436237e+04 1.3e+00 2.33e-02  2e-02  2e-02 0:10.7
  400   6800 -1.905064268546563e+04 1.4e+00 4.14e-03  4e-03  4e-03 0:14.2
  500   8500 -1.905082182666092e+04 1.4e+00 8.60e-04  7e-04  9e-04 0:17.8
  600  10200 -1.905083027208606e+04 1.4e+00 2.18e-04  2e-04  2e-04 0:21.3
  700  11900 -1.905083081825022e+04 1.7e+00 6.19e-05  5e-05  8e-05 0:24.9
  800  13600 -1.905083088164421e+04 2.1e+00 2.00e-05  2e-05  3e-05 0:28.4
  900  15300 -1.905083089070728e+04 2.7e+00 9.60e-06  7e-06  2e-05 0:31.9
 1000  17000 -1.905083089276799e+04 3.3e+00 5.08e-06  4e-06  1e-05 0:35.5
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  18700 -1.905083089318332e+04 3.8e+00 2.23e-06  2e-06  6e-06 0:39.0
 1200  20400 -1.905083089323065e+04 4.0e

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [12551.07538939] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=12551.075389, sigma=2.40e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=12551.075389, sigma=2.40e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  '

    2     34 4.677693176885703e+03 1.0e+00 2.33e+00  2e+00  2e+00 0:00.1
    3     51 5.515511028460987e+03 1.0e+00 2.27e+00  2e+00  2e+00 0:00.1


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [4677.69317689] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


   79   1343 -1.165107375088839e+03 1.3e+00 1.60e+00  2e+00  2e+00 0:03.1
  100   1700 -9.818100416652760e+02 1.3e+00 1.32e+00  1e+00  1e+00 0:04.0
  200   3400 -6.452978042723131e+03 1.4e+00 4.84e-01  4e-01  5e-01 0:08.0
  300   5100 -8.162705597313182e+03 1.6e+00 2.54e-01  2e-01  3e-01 0:12.0
  400   6800 -8.668474234220539e+03 1.7e+00 1.07e-01  9e-02  1e-01 0:16.0
  500   8500 -8.836434923579312e+03 2.0e+00 7.20e-02  6e-02  8e-02 0:20.0
  600  10200 -8.917141160351452e+03 2.4e+00 5.03e-02  4e-02  6e-02 0:24.0
  700  11900 -8.988292367122242e+03 3.5e+00 5.63e-02  5e-02  1e-01 0:28.0
  800  13600 -9.016172132614360e+03 3.8e+00 2.71e-02  2e-02  5e-02 0:32.0
  900  15300 -9.022529748492761e+03 3.8e+00 1.59e-02  1e-02  3e-02 0:36.0
 1000  17000 -9.027783949047847e+03 4.0e+00 1.54e-02  1e-02  3e-02 0:39.9
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  18700 -9.063105596620419e+03 6.3e+00 4.87e-02  4e-02  2e-01 0:43.9
 1200  20400 -9.107749046833265e+03 8.1e

In [7]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('Results\\F3_X_Values.csv')
Krig_Fun = np.zeros(30)
SVM_Fun = np.zeros(30)
ELN_Fun = np.zeros(30)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F3(X_Values.iloc[i,:100])
    SVM_Fun [i] = F3(X_Values.iloc[i,100:200])
    ELN_Fun [i] = F3(X_Values.iloc[i,200:300])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([1900.23671272]), count=array([1])) 7.509950880554293e-05
SVM
ModeResult(mode=array([9521.50307912]), count=array([1])) 3667.079987703804
ELN
ModeResult(mode=array([5469.49522809]), count=array([1])) 511.61452749786366
